This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_residuals_histogram(y_true, y_pred, savepath,
                             stats, title='residuals histogram', label='residuals'):
    """
    Method to calculate and plot the histogram of residuals from regression model

    Args:

        y_true: (numpy array), array of true y data values

        y_pred: (numpy array), array of predicted y data values

        savepath: (str), path to save the plotted precision-recall curve

        stats: (dict), dict of training or testing statistics for a particular run

        title: (str), title of residuals histogram

        label: (str), label used for axis labeling

    Returns:

        None

    """

    # make fig and ax, use x_align when placing text so things don't overlap
    x_align = 0.64
    fig, ax = make_fig_ax(x_align=x_align)

    #ax.set_title(title)
    # do the actual plotting
    residuals = y_true - y_pred

    #Output residuals data and stats to spreadsheet
    path = os.path.dirname(savepath)
    pd.DataFrame(residuals).describe().to_csv(os.path.join(path,'residual_statistics.csv'))
    pd.DataFrame(residuals).to_csv(path+'/'+'residuals.csv')

    #Get num_bins using smarter method
    num_bins = get_histogram_bins(y_df=residuals)
    ax.hist(residuals, bins=num_bins, color='b', edgecolor='k')

    # normal text stuff
    ax.set_xlabel('Value of '+label, fontsize=16)
    ax.set_ylabel('Number of occurences', fontsize=16)

    # make y axis ints, because it is discrete
    #ax.yaxis.set_major_locator(MaxNLocator(integer=True))

    plot_stats(fig, stats, x_align=x_align, y_align=0.90)
    plot_stats(fig, pd.DataFrame(residuals).describe().to_dict()[0], x_align=x_align, y_align=0.60)

    fig.savefig(savepath, dpi=DPI, bbox_inches='tight')


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
y_true = array([2.02092457e+02, 2.45647989e+02, 2.19470531e+02, 2.09634413e+02,
       3.40489474e+02, 3.88395179e+02, 2.59721126e+02, 1.40876460e+02,
       1.10462825e+02, 9.45736257e+01, 9.62792101e+01, 9.53431111e+01,
       2.41991216e+02, 1.45936466e+02, 3.04216098e+02, 1.23826343e+02,
       1.77346326e+02, 8.94641218e+01, 0.00000000e+00, 3.21795003e+02,
       1.70828354e+02, 1.95159840e+02, 1.96095876e+02, 1.37316410e+02,
       3.50019597e+02, 3.02985863e+02, 3.19706165e+02, 2.27914069e+02,
       8.44124962e+01, 1.03964994e+02, 2.09882629e+02, 7.02166732e+01,
       2.16105345e+01, 2.89476692e+01, 6.30457358e+01, 3.87490899e+01,
       2.56639133e+01, 7.66050637e+01, 1.18303763e+02, 4.17751932e+02,
       1.29128414e+02, 1.00971120e+02, 6.36339325e+02, 6.86792014e+01,
       2.72804393e+02, 1.78235739e+02, 2.27491285e+02, 1.22311356e+02,
       2.74349274e+02, 2.78307782e+02, 1.82740912e+02, 0.00000000e+00,
       9.04068336e+01, 7.58782445e+01, 6.92869614e+01, 1.29458568e+01,
       5.42920320e+01, 3.57160729e+02, 4.00299318e+01, 2.68896498e+01,
       0.00000000e+00, 7.80236106e+01, 1.16763996e+00, 0.00000000e+00,
       0.00000000e+00, 6.61459738e+01, 8.03088872e+01, 1.77441804e+02,
       9.68586984e+01, 7.03541024e+01, 1.27239967e+02, 2.53560446e+02,
       2.51771744e+02, 5.95521104e+01, 4.11709055e+01, 4.52159618e+01,
       0.00000000e+00, 1.66684097e+01, 0.00000000e+00, 2.03972667e-01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       7.39741468e+01, 3.20782685e+01, 1.54429064e+01, 1.73373103e+01,
       7.53897668e+01, 0.00000000e+00, 3.49408401e+01, 6.03605982e+01,
       3.27899307e+01, 6.05219124e+01, 2.67022126e+01, 0.00000000e+00,
       4.42073704e-01, 1.78001455e+02, 1.81910775e+02, 1.31304680e+02,
       1.14247314e+02, 9.92490625e-01, 4.93632300e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.48100516e+01,
       4.23248061e+01, 0.00000000e+00, 2.57222520e+01, 2.21842737e+01,
       2.08989075e+02, 5.38520339e+01, 3.13780640e+01, 3.66852424e+01,
       3.51232216e+01, 3.51583952e+01, 7.30016409e+01, 1.05693165e+02,
       7.80094958e+01, 1.32109024e+02, 7.79804428e+01, 1.23754016e+02,
       2.68325976e+01, 0.00000000e+00, 2.78984665e+02, 2.24784003e+01,
       5.91838239e+01, 1.25292208e+02, 6.99723855e+01, 4.70862505e+01,
       9.34258071e+01, 1.14554644e+02, 1.58427043e+02, 1.09033411e+02,
       2.57075963e+02, 2.63839575e+02, 2.76340269e+02, 8.39171957e+01,
       0.00000000e+00, 0.00000000e+00, 2.80876470e+02, 2.11460716e+02,
       2.81102443e+02, 2.86014919e+02, 2.02283948e+02, 2.30921905e+02,
       1.28448866e+02, 5.08392287e+01, 2.91802070e+01, 1.31808017e+01,
       0.00000000e+00, 0.00000000e+00, 2.93008151e+01, 0.00000000e+00,
       0.00000000e+00, 1.21655324e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.84401941e+01, 5.12388154e+01,
       3.08078053e+01, 4.81001678e+01, 4.45494170e+01, 2.34800709e+02,
       7.54658121e+01, 8.29266298e+01, 6.06316356e+01, 4.63304068e+01,
       1.10052006e+02, 9.70888906e+01, 5.13090164e+01, 5.33902792e+01,
       4.43244216e+01, 0.00000000e+00, 3.92079232e+01, 1.54697467e+02,
       1.02151271e+02, 8.15909270e+01, 9.38711282e+01, 0.00000000e+00,
       2.64912691e+01, 4.00673869e+01, 4.95815034e+01, 6.33172566e+01,
       1.02395743e+02, 3.11645320e+01, 1.76733958e+01, 0.00000000e+00,
       2.75959946e+01, 1.40515010e+02, 4.27492462e+01, 5.19507218e+01,
       1.01745118e+01, 1.92994335e+01, 0.00000000e+00, 9.40584552e+01,
       1.05442368e+02, 6.89012756e+00, 2.32118071e+01, 6.90074178e+01,
       0.00000000e+00, 2.61974495e+01, 0.00000000e+00, 3.56929084e+01,
       1.20413789e+02, 1.55663046e+02, 1.93229456e+02, 1.83828681e+02,
       3.42040019e+02, 3.66503081e+02, 2.76006928e+02, 9.15340044e+01,
       1.13869162e+02, 1.18103266e+02, 2.20003381e+02, 1.39013581e+02,
       2.59886084e+02, 2.85344428e+02, 1.44708713e+02, 1.44006558e+02,
       1.90075255e+02, 8.30462436e+01, 1.21428977e+02, 2.31481855e+02,
       1.77752168e+02, 2.51196620e+02, 1.54419624e+02, 2.36338029e+02,
       2.50579269e+02, 2.67340580e+02, 2.62896301e+02, 2.21055904e+02,
       3.40306363e+02, 1.89648365e+02, 2.33810236e+02, 3.37214376e+02,
       1.87894409e+02, 5.98511381e+01, 1.80660958e+02, 1.10209088e+02,
       9.69915119e+01, 1.49647778e+02, 1.93969856e+02, 1.00763646e+02,
       1.88513164e+02, 1.31977595e+02, 1.02794584e+02, 1.21127399e+02,
       2.53019864e+02, 3.32212137e+02, 2.52001050e+02, 3.17356871e+02,
       2.98525793e+02, 3.23634151e+02, 2.23307827e+02, 3.18434650e+02,
       8.54936441e+01, 7.99859482e+01, 1.34599602e+02, 1.48873887e+02,
       7.93248059e+01, 1.09381016e+02, 6.02850907e+01, 5.55045542e+01,
       1.50801863e+02, 1.60434807e+02, 1.80035168e+02, 2.72871580e+02,
       2.02525534e+02, 2.10329787e+02, 2.99112578e+02, 1.44135045e+02,
       1.73746536e+02, 1.83572992e+02, 2.04542101e+02, 2.71906598e+02,
       1.51155130e+02, 1.10472455e+02, 1.35426489e+02, 9.88049065e+01,
       1.07655170e+02, 8.10050964e+01, 9.27064680e+01, 9.69622803e+01,
       1.31798616e+02, 1.05836151e+02, 1.08312182e+02, 1.89420372e+02,
       1.22185191e+02, 1.02217058e+02, 1.10480194e+02, 6.28478449e+01,
       4.68826690e+01, 2.72145096e+01, 9.31208466e+01, 4.43281812e+01,
       6.20242803e+01, 4.34380611e+01, 1.25171931e+02, 1.82785945e+01,
       2.20249128e+01, 5.66837962e+01, 7.58528445e+01, 1.31671414e+02,
       5.12762345e+01, 1.40218828e+02, 1.78369049e+02, 1.84215977e+02,
       1.13527293e+02, 1.85414463e+02, 1.63793911e+02, 1.42933388e+02,
       1.84787732e+02, 1.59787224e+02, 1.47483468e+02, 1.63982208e+02,
       2.98870605e+01, 1.05722180e+02, 1.09488956e+02, 9.60360384e+01,
       1.26798490e+02, 1.04372971e+02, 1.04244208e+02, 1.88514713e+02,
       1.85051275e+02, 1.15594393e+02, 1.09297300e+02, 1.62419081e+02,
       1.37369485e+02, 1.91314596e+02, 6.97752946e+01, 2.97477073e+01,
       3.71426186e+01, 3.63667394e+01, 3.48221974e+01, 1.97610067e+02,
       2.15562064e+01, 7.10907636e+01, 8.71299483e+01, 1.91667256e+01,
       5.97413320e+01, 3.97988019e+01, 5.91429461e+01, 1.98665047e+02,
       1.34244899e+02, 3.52832986e+01, 9.38114347e+01, 9.98669764e+01,
       1.43739984e+02, 3.81360533e+01, 1.23689710e+02, 0.00000000e+00,
       2.41527621e+01, 0.00000000e+00, 0.00000000e+00, 5.32856309e+01,
       4.94793928e+01, 7.26725204e+01, 1.00472922e+02, 7.32347672e+01,
       3.97951736e+01, 0.00000000e+00, 4.87758689e+01, 1.60849161e+02,
       4.34643306e+01, 6.39648556e+01, 2.12813897e+01, 2.63539598e+01,
       3.38853640e+01, 2.35812529e+01, 3.73769471e+01, 7.48181406e+01,
       0.00000000e+00, 3.40153297e+01, 2.67854426e+02, 2.07757183e+02,
       8.29965720e+01, 7.72377141e+01, 1.52154945e+02, 6.98177552e+01,
       5.51408019e+01, 6.05410832e+01, 8.22454137e+01, 6.94738840e+01,
       4.54995709e+01, 4.38219567e+01, 4.23854981e+01, 3.83815528e+01,
       0.00000000e+00, 0.00000000e+00, 1.77019953e+01, 7.17619802e+01,
       3.15229082e+01, 0.00000000e+00, 9.46025620e+01, 8.48759224e+01,
       1.42323884e+02, 3.74352793e+01, 3.12618672e+01, 2.59562148e+01,
       8.01561257e+01, 5.19044272e+01, 3.87785474e+01, 5.43341092e+01,
       1.99708105e+01, 6.57797960e+01, 2.51156351e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.88957798e+01,
       4.72079029e+01])
y_pred = array([0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 3.16810129e+001, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 6.11556778e+001,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 9.62095983e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       2.12969881e-200, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 5.58332385e-200, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 1.56210203e+001,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 1.47151242e-316, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000])
savepath = '/home/nerve/Desktop/skunkworks/lane_schultz/hyper_parameters/test/DoNothing/DoNothing/KernelRidge_DoNothing_DoNothing_GridSearch/RepeatedKFold/split_3/test_residuals_histogram.png'
stats = OrderedDict([('R2', 9.933753881052888e-05), ('root_mean_squared_error', 140.9579839940036), ('mean_absolute_error', 104.21199898371502), ('rmse_over_stdev', 1.4758091945050607)])
title = 'test_residuals_histogram'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_residuals_histogram(y_true, y_pred, savepath, stats, title, label)
display(Image(filename='test_residuals_histogram.png'))
